This code is built for the analysis of patches from a single WSI. 

To iterate over the multiple, paths should be adjusted respectively. 

The original folder consists of x_10.png, x_20.png, x_40.png, x_20_generated.png, x_40_generated.png patches. 

This script organizes generated folders into respective different folders and then calculate the metrics for quality assesment of generated images

In [15]:
import os
import shutil

# path to randomly selected slide ID 
folder_path = '/Users/idilunlu/Desktop/upscaler/8624293b-7775-11eb-9a27-001a7dda7111' 
# Update this with your actual folder path

destination_20 = os.path.join(folder_path, '20_files')
destination_40 = os.path.join(folder_path, '40_files')

os.makedirs(destination_20, exist_ok=True)
os.makedirs(destination_40, exist_ok=True)

# Loop through the files 
for filename in os.listdir(folder_path):
    if filename.endswith("20_generated.png"):
        base_name = filename.replace('_20_generated.png', '')
        # Copy the _20_generated.png file
        shutil.copy(os.path.join(folder_path, filename), os.path.join(destination_20, filename))
        # Copy the corresponding _20.png file if it exists
        corresponding_file = base_name + '_20.png'
        if os.path.exists(os.path.join(folder_path, corresponding_file)):
            shutil.copy(os.path.join(folder_path, corresponding_file), os.path.join(destination_20, corresponding_file))
    elif filename.endswith("40_generated.png"):
        base_name = filename.replace('_40_generated.png', '')
        # Copy the _40_generated.png file
        shutil.copy(os.path.join(folder_path, filename), os.path.join(destination_40, filename))
        # Copy the corresponding _40.png file if it exists
        corresponding_file = base_name + '_40.png'
        if os.path.exists(os.path.join(folder_path, corresponding_file)):
            shutil.copy(os.path.join(folder_path, corresponding_file), os.path.join(destination_40, corresponding_file))

print("Files have been copied to the respective folders.")


Files have been copied to the respective folders.


In [17]:
# Set the directory path where the files are located
directory_path = '/Users/idilunlu/Desktop/upscaler/8624293b-7775-11eb-9a27-001a7dda7111/20_files'

# Iterate over the files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith("_20.png") and not filename.endswith("_20_generated.png"):
        # Construct the full path to the file
        file_path = os.path.join(directory_path, filename)
        # Delete the file
        os.remove(file_path)
        print(f"Deleted {filename}")


Deleted 317_20.png
Deleted 446_20.png
Deleted 217_20.png
Deleted 546_20.png
Deleted 1195_20.png
Deleted 1473_20.png
Deleted 1322_20.png
Deleted 899_20.png
Deleted 1095_20.png
Deleted 1573_20.png
Deleted 1222_20.png
Deleted 999_20.png
Deleted 309_20.png
Deleted 458_20.png
Deleted 209_20.png
Deleted 558_20.png
Deleted 1410_20.png
Deleted 1341_20.png
Deleted 1510_20.png
Deleted 1241_20.png
Deleted 692_20.png
Deleted 374_20.png
Deleted 425_20.png
Deleted 82_20.png
Deleted 792_20.png
Deleted 274_20.png
Deleted 525_20.png
Deleted 887_20.png
Deleted 987_20.png
Deleted 554_20.png
Deleted 205_20.png
Deleted 454_20.png
Deleted 305_20.png
Deleted 1230_20.png
Deleted 1561_20.png
Deleted 1087_20.png
Deleted 1330_20.png
Deleted 1461_20.png
Deleted 1187_20.png
Deleted 529_20.png
Deleted 278_20.png
Deleted 429_20.png
Deleted 378_20.png
Deleted 1253_20.png
Deleted 1502_20.png
Deleted 1353_20.png
Deleted 1402_20.png
Deleted 1199_20.png


In [11]:
import numpy as np
from skimage import io, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from scipy import linalg
from scipy.spatial import distance
import torch
import torchvision.transforms as transforms
from torchvision.models import inception_v3
import torch.nn.functional as F
from PIL import Image

# Paths to image folders
original_folder = '/Users/idilunlu/Desktop/upscaler/8624293b-7775-11eb-9a27-001a7dda7111'  # Update this with your actual path
generated_folder_20 = '/Users/idilunlu/Desktop/upscaler/8624293b-7775-11eb-9a27-001a7dda7111/20_files'    # Update this with your actual path
generated_folder_40 = '/Users/idilunlu/Desktop/upscaler/8624293b-7775-11eb-9a27-001a7dda7111/40_files'    # Update this with your actual path

### Performance Metrics Calculator

In [13]:
# Function to resize images
def resize_image(image, size):
    if size[0] < MIN_SIZE or size[1] < MIN_SIZE:
        size = (MIN_SIZE, MIN_SIZE)
    image_pil = Image.fromarray((image * 255).astype(np.uint8))
    resized_image_pil = image_pil.resize(size, Image.BILINEAR)
    return np.array(resized_image_pil) / 255.0

# Calculate MSE
def mse(imageA, imageB):
    return np.mean((imageA - imageB) ** 2)

# PSNR Calculation
def calculate_psnr(mse, max_pixel_value=1.0):
    if mse == 0:
        return float('inf')
    return 10 * np.log10(max_pixel_value ** 2 / mse)

# Calculate PSNR and MSE
def calculate_metrics(original_folder, generated_folder):
    psnr_scores = []
    mse_scores = []

    for file in os.listdir(generated_folder):
        original_file = os.path.join(original_folder, file.replace('_generated', ''))
        generated_file = os.path.join(generated_folder, file)
        if os.path.exists(original_file) and os.path.exists(generated_file):
            original_img = img_as_float(io.imread(original_file))
            generated_img = img_as_float(io.imread(generated_file))

            # Ensure images are in the correct format
            if original_img.ndim == 2:
                original_img = np.stack([original_img]*3, axis=-1)
            if generated_img.ndim == 2:
                generated_img = np.stack([generated_img]*3, axis=-1)

            # Resize images to the same dimensions
            min_shape = (min(original_img.shape[0], generated_img.shape[0]),
                         min(original_img.shape[1], generated_img.shape[1]))
            original_img = resize_image(original_img, min_shape)
            generated_img = resize_image(generated_img, min_shape)

            # Debugging statements to print shapes
            # print(f"Original image shape after resizing: {original_img.shape}")
            # print(f"Generated image shape after resizing: {generated_img.shape}")

            mse_score = mse(original_img, generated_img)
            psnr_score = calculate_psnr(mse_score)

            psnr_scores.append(psnr_score)
            mse_scores.append(mse_score)

    return {
        'PSNR': np.mean(psnr_scores),
        'MSE': np.mean(mse_scores)
    }

metrics_20 = calculate_metrics(original_folder, generated_folder_20)
metrics_40 = calculate_metrics(original_folder, generated_folder_40)

print("Metrics for 20 generated images:", metrics_20)
print("Metrics for 40 generated images:", metrics_40)

Metrics for 20 generated images: {'PSNR': 24.4357573994683, 'MSE': 0.0037269711688408392}
Metrics for 40 generated images: {'PSNR': 23.186037926194654, 'MSE': 0.004999256733550635}


In [14]:
def calculate_ssim(original_folder, generated_folder):
    ssim_scores = []

    for file in os.listdir(generated_folder):
        original_file = os.path.join(original_folder, file.replace('_generated', ''))
        generated_file = os.path.join(generated_folder, file)
        if os.path.exists(original_file) and os.path.exists(generated_file):
            original_img = img_as_float(io.imread(original_file))
            generated_img = img_as_float(io.imread(generated_file))

            # Ensure images are in the correct format
            if original_img.ndim == 2:
                original_img = np.stack([original_img]*3, axis=-1)
            if generated_img.ndim == 2:
                generated_img = np.stack([generated_img]*3, axis=-1)

            # Resize images to the same dimensions
            min_shape = (min(original_img.shape[0], generated_img.shape[0]),
                         min(original_img.shape[1], generated_img.shape[1]))
            original_img = resize_image(original_img, min_shape)
            generated_img = resize_image(generated_img, min_shape)

            # Debugging statements to print shapes and values
            print(f"Original image shape after resizing: {original_img.shape}")
            print(f"Generated image shape after resizing: {generated_img.shape}")

            ssim_score = ssim(original_img, generated_img, multichannel=True)

            # Additional debugging information
            print(f"SSIM for {file}: {ssim_score}")

            ssim_scores.append(ssim_score)

    return {
        'SSIM': np.mean(ssim_scores)
    }

# Calculate and print SSIM metrics for 20 and 40 generated images
metrics_ssim_20 = calculate_ssim(original_folder, generated_folder_20)
metrics_ssim_40 = calculate_ssim(original_folder, generated_folder_40)

print("SSIM for 20 generated images:", metrics_ssim_20)
print("SSIM for 40 generated images:", metrics_ssim_40)

Original image shape after resizing: (224, 224, 3)
Generated image shape after resizing: (224, 224, 3)


ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.